# 1. Import required Packages:

In [1]:
import pandas as pd
import io
import requests
import datetime
import difflib
import numpy as np

In [2]:
pd.set_option("display.max_rows", 100, "display.max_columns", None)
pd.set_option('precision', 1)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## We're going to use date made available by John Hopkins University on https://github.com/CSSEGISandData

## They have produced timeseries for confirmed cases, deaths and recovered measures for all countries in World Health Organisation (WHO) data and supplied the daily reports.

## Getting the date automatically so that we can always pull a recent daily report.

## In order to avoid errors, we can use  today's date -1 (because the data is updated in the USA and may be pulled from other timezones, eg: Australia).

In [3]:
# create a datetime object for yesterday's date
d = datetime.datetime.today() - datetime.timedelta(days=2)
# create a string that matches the format of John Hopkins' data
yesterday_date = d.strftime('%m-%d-%Y')+".csv"

## Getting the data:

In [4]:
# The population mortatlity data is taken from a business insider article about south korean fatalitites by age , each value is a percentage

mortality_by_age = [0, 0.1, 0.1, 0.4, 1.5, 4.3, 7.2]

In [5]:
# function to scrape data from a url with a csv file
def scrape(url):
    pull = requests.get(url).content
    data = pd.read_csv(io.StringIO(pull.decode('utf-8')))
    return data

In [6]:
# urls for the data
johnhopkins_confirmed_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
johnhopkins_deaths_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
johnhopkins_recovered_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
johnhopkins_daily_reports = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports/" + yesterday_date

# scraping data
confirmed = scrape(johnhopkins_confirmed_url)
deaths = scrape(johnhopkins_deaths_url)
recovered = scrape(johnhopkins_recovered_url)
daily = scrape(johnhopkins_daily_reports)

In [7]:
# getting demographic and hospital bed data from local files
demographics = pd.read_csv('world_demographics_filtered.csv')
hospital_beds = pd.read_csv('hosp_beds_data.csv')

## Inspecting the data and structure:

In [8]:
print('confirmed shape:', confirmed.shape)
confirmed.head()

confirmed shape: (264, 90)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,NaN,Afghanistan,33.00,65.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521,555,607,665,714,784,840
1,NaN,Albania,41.15,20.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518
2,NaN,Algeria,28.03,1.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268
3,NaN,Andorra,42.51,1.52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673
4,NaN,Angola,-11.20,17.87,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19


In [9]:
print('deaths shape:', deaths.shape)
deaths.head()

deaths shape: (264, 90)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,NaN,Afghanistan,33.00,65.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,4,4,4,4,4,4,4,6,6,7,7,11,14,14,15,15,18,18,21,23,25,30
1,NaN,Albania,41.15,20.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,4,5,5,6,8,10,10,11,15,15,16,17,20,20,21,22,22,23,23,23,23,23,24,25,26
2,NaN,Algeria,28.03,1.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,4,4,4,7,9,11,15,17,17,19,21,25,26,29,31,35,44,58,86,105,130,152,173,193,205,235,256,275,293,313,326,336,348
3,NaN,Andorra,42.51,1.52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3,3,3,6,8,12,14,15,16,17,18,21,22,23,25,26,26,29,29,31,33,33
4,NaN,Angola,-11.20,17.87,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [10]:
print('recovered shape:', recovered.shape)
recovered.head()

recovered shape: (250, 90)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,NaN,Afghanistan,33.00,65.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,5,5,10,10,10,15,18,18,29,32,32,32,32,32,40,43,54
1,NaN,Albania,41.15,20.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44,52,67,76,89,99,104,116,131,154,165,182,197,217,232,248,251,277
2,NaN,Algeria,28.03,1.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,12,12,12,12,12,32,32,32,65,65,24,65,29,29,31,31,37,46,61,61,62,90,90,90,113,237,347,405,460,591,601,691,708,783
3,NaN,Andorra,42.51,1.52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,10,10,10,10,16,21,26,31,39,52,58,71,71,128,128,128,169,169
4,NaN,Angola,-11.20,17.87,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,2,2,2,2,4,4,4,5,5,5


In [11]:
print('daily shape:', daily.shape)
daily.head()
daily.Country_Region.value_counts()
daily.to_csv("daily.csv")

daily shape: (3027, 12)


In [12]:
print('demographics shape:', demographics.shape)
demographics.head()

demographics shape: (21053, 9)


,Country or Area,Year,Area,Sex,Age,Record Type,Reliability,Source Year,Value
0,Albania,2018,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2019,2870324.00
1,Albania,2018,Total,Both Sexes,0,Estimate - de jure,"Final figure, complete",2019,30867.00
2,Albania,2018,Total,Both Sexes,0 - 4,Estimate - de jure,"Final figure, complete",2019,162416.00
3,Albania,2018,Total,Both Sexes,1,Estimate - de jure,"Final figure, complete",2019,31137.00
4,Albania,2018,Total,Both Sexes,1-Apr,Estimate - de jure,"Final figure, complete",2019,131549.00


Dropping columns with information we don't need from the demographics dataframe

In [13]:
print('hosp_beds shape:', hospital_beds.shape)
hospital_beds.head()

hosp_beds shape: (181, 3)


,Country,Year,Hospital beds (per 10 000 population)
0,Afghanistan,2015,5
1,Albania,2013,29
2,Algeria,2015,19
3,Angola,2005,8
4,Antigua and Barbuda,2014,38


In [14]:
demographics = demographics.drop(columns=['Area', 'Sex', 'Record Type', 'Reliability','Source Year'])
demographics.head()

,Country or Area,Year,Age,Value
0,Albania,2018,Total,2870324.00
1,Albania,2018,0,30867.00
2,Albania,2018,0 - 4,162416.00
3,Albania,2018,1,31137.00
4,Albania,2018,1-Apr,131549.00


## As there are multiple record types, we should drop duplicate age ranges from different record types to avoid double counting.

In [15]:
demographics= demographics.drop_duplicates(subset=['Country or Area', 'Age', 'Year'], keep='first')

## For some reason, some of the 'Age' data has screwed up ranges. The csv file has 4-Sep instead of 4-9 (a date issue seemingly, as september is the 9th month) We should identify these values and replace them with correct values.

In [16]:
# identify data age data containing non age related strings
demographics.Age.unique()

array(['Total', '0', '0 - 4', '1', '1-Apr', '2', '3', '4', '5', '5-Sep',
       '6', '7', '8', '9', '10', 'Oct-14', '11', '12', '13', '14', '15',
       '15 - 19', '16', '17', '18', '19', '20', '20 - 24', '21', '22',
       '23', '24', '25', '25 - 29', '26', '27', '28', '29', '30',
       '30 - 34', '31', '32', '33', '34', '35', '35 - 39', '36', '37',
       '38', '39', '40', '40 - 44', '41', '42', '43', '44', '45',
       '45 - 49', '46', '47', '48', '49', '50', '50 - 54', '51', '52',
       '53', '54', '55', '55 - 59', '56', '57', '58', '59', '60',
       '60 - 64', '61', '62', '63', '64', '65', '65 - 69', '66', '67',
       '68', '69', '70', '70 - 74', '71', '72', '73', '74', '75',
       '75 - 79', '76', '77', '78', '79', '80', '80 - 84', '81', '82',
       '83', '84', '85 +', '85', '85 - 89', '86', '87', '88', '89', '90',
       '90 - 94', '91', '92', '93', '94', '95', '95 - 99', '96', '97',
       '98', '99', '100', '100 +', '95 +', '75 +', '80 +', 'Unknown',
       '90 +', '100 

In [17]:
# replacing age ranges that the csv has output as dates
demographics['Age'] = demographics["Age"].replace('1-Apr', '1-4')
demographics['Age'] = demographics["Age"].replace('5-Sep', '5-9')
demographics['Age'] = demographics["Age"].replace('Oct-14', '10-14')
demographics['Age'] = demographics["Age"].replace('Jul-14', '7-14')
demographics['Age'] = demographics["Age"].replace('Oct-19', '10-19')
demographics['Age'] = demographics["Age"].replace('May-14', '5-14')
demographics['Age'] = demographics["Age"].replace('Nov-20', '11-20')
demographics['Age'] = demographics["Age"].replace('Jan-14', '1-14')
demographics['Age'] = demographics["Age"].replace('6-Oct', '6-10')
demographics['Age'] = demographics["Age"].replace('Nov-15', '11-15')

In [18]:
# find values we want to drop and drop them
searchfor = ['Unknown']
demographics= demographics[~demographics['Age'].str.contains('|'.join(searchfor))]
demographics.Age.unique()

array(['Total', '0', '0 - 4', '1', '1-4', '2', '3', '4', '5', '5-9', '6',
       '7', '8', '9', '10', '10-14', '11', '12', '13', '14', '15',
       '15 - 19', '16', '17', '18', '19', '20', '20 - 24', '21', '22',
       '23', '24', '25', '25 - 29', '26', '27', '28', '29', '30',
       '30 - 34', '31', '32', '33', '34', '35', '35 - 39', '36', '37',
       '38', '39', '40', '40 - 44', '41', '42', '43', '44', '45',
       '45 - 49', '46', '47', '48', '49', '50', '50 - 54', '51', '52',
       '53', '54', '55', '55 - 59', '56', '57', '58', '59', '60',
       '60 - 64', '61', '62', '63', '64', '65', '65 - 69', '66', '67',
       '68', '69', '70', '70 - 74', '71', '72', '73', '74', '75',
       '75 - 79', '76', '77', '78', '79', '80', '80 - 84', '81', '82',
       '83', '84', '85 +', '85', '85 - 89', '86', '87', '88', '89', '90',
       '90 - 94', '91', '92', '93', '94', '95', '95 - 99', '96', '97',
       '98', '99', '100', '100 +', '95 +', '75 +', '80 +', '90 +',
       '100 - 104', '101', '

## We can merge the daily and demographics datasets, but first we need to normalize the country names in each dataframe and consolidate the age ranges in demographics

## Consolidating names:

In [19]:
# changing names of daily to be more compact
daily['Country_Region'] = daily["Country_Region"].replace("United Kingdom", "UK")
daily['Country_Region'] = daily["Country_Region"].replace("Bosnia and Herzegovina","Bosnia & H.")
daily['Country_Region'] = daily["Country_Region"].replace("West Bank and Gaza","West Bank/Gaza")
daily['Country_Region'] = daily["Country_Region"].replace("North Macedonia","N. Macedonia")
daily['Country_Region'] = daily["Country_Region"].replace("Trinidad and Tobago","Trinidad & T.")
daily['Country_Region'] = daily["Country_Region"].replace("Korea, South","South Korea")
daily['Country_Region'] = daily["Country_Region"].replace("Antigua and Barbuda","Antigua & B.")
daily['Country_Region'] = daily["Country_Region"].replace("Saint Vincent and the Grenadines","St Vincent & G.")
daily['Country_Region'] = daily["Country_Region"].replace("Saint Kitts and Nevis",'St Kitts')
daily['Country_Region'] = daily["Country_Region"].replace("Dominican Republic",'Dominican Rep')
daily['Country_Region'] = daily["Country_Region"].replace("Sao Tome and Principe",'Sao Tome & P.')
daily['Country_Region'] = daily["Country_Region"].replace('Congo (Brazzaville)','Congo')
daily['Country_Region'] = daily["Country_Region"].replace('United Arab Emirates','UAE')
daily['Country_Region'] = daily["Country_Region"].replace('Papua New Guinea','Papua N.G.')

# changing names in demographics dataframe to match daily dataframe
demographics['Country or Area'] = demographics["Country or Area"].replace('Bolivia (Plurinational State of)', 'Bolivia')
demographics["Country or Area"] = demographics["Country or Area"].replace("Viet Nam", "Vietnam")
demographics["Country or Area"] = demographics["Country or Area"].replace("United States of America", "US")
demographics["Country or Area"] = demographics["Country or Area"].replace("United Kingdom of Great Britain and Northern Ireland", "UK")
demographics["Country or Area"] = demographics["Country or Area"].replace("Republic of Korea", "South Korea")
demographics["Country or Area"] = demographics["Country or Area"].replace("Venezuela (Bolivarian Republic of)", "Venezuela")
demographics["Country or Area"] = demographics["Country or Area"].replace('Iran (Islamic Republic of)', 'Iran')
demographics["Country or Area"] = demographics["Country or Area"].replace('Russian Federation', 'Russia')
demographics["Country or Area"] = demographics["Country or Area"].replace('Brunei Darussalam', 'Brunei')
demographics["Country or Area"] = demographics["Country or Area"].replace('Republic of Moldova', 'Moldova')
demographics["Country or Area"] = demographics["Country or Area"].replace('United Republic of Tanzania', 'Tanzania')
demographics["Country or Area"] = demographics["Country or Area"].replace("Lao People's Democratic Republic", 'Laos')
demographics["Country or Area"] = demographics["Country or Area"].replace("Myanmar", 'Burma')
demographics["Country or Area"] = demographics["Country or Area"].replace("Congo", 'Congo')
demographics["Country or Area"] = demographics["Country or Area"].replace("Côte d'Ivoire", "Cote d'Ivoire")
demographics["Country or Area"] = demographics["Country or Area"].replace("State of Palestine", "West Bank/Gaza")
demographics['Country or Area'] = demographics["Country or Area"].replace("United Kingdom", "UK")
demographics['Country or Area'] = demographics["Country or Area"].replace("Bosnia and Herzegovina","Bosnia & H.")
demographics['Country or Area'] = demographics["Country or Area"].replace("West Bank and Gaza","West Bank/Gaza")
demographics['Country or Area'] = demographics["Country or Area"].replace("North Macedonia","N. Macedonia")
demographics['Country or Area'] = demographics["Country or Area"].replace("Trinidad and Tobago","Trinidad & T.")
demographics['Country or Area'] = demographics["Country or Area"].replace("Korea, South","South Korea")
demographics['Country or Area'] = demographics["Country or Area"].replace("Antigua and Barbuda","Antigua & B.")
demographics['Country or Area'] = demographics["Country or Area"].replace("Saint Vincent and the Grenadines","St Vincent & G.")
demographics['Country or Area'] = demographics["Country or Area"].replace("Saint Kitts and Nevis",'St Kitts')
demographics['Country or Area'] = demographics["Country or Area"].replace("Dominican Republic",'Dominican Rep')
demographics['Country or Area'] = demographics["Country or Area"].replace("Sao Tome and Principe",'Sao Tome & P.')
demographics['Country or Area'] = demographics["Country or Area"].replace('Congo (Brazzaville)','Congo')
demographics['Country or Area'] = demographics["Country or Area"].replace('United Arab Emirates','UAE')
demographics['Country or Area'] = demographics["Country or Area"].replace('Papua New Guinea','Papua N.G.')


# The hospital beds have the same naming convention, so should be changes the same way
hospital_beds['Country'] = hospital_beds["Country"].replace('Bolivia (Plurinational State of)', 'Bolivia')
hospital_beds["Country"] = hospital_beds["Country"].replace("Viet Nam", "Vietnam")
hospital_beds["Country"] = hospital_beds["Country"].replace("United States of America", "US")
hospital_beds["Country"] = hospital_beds["Country"].replace("United Kingdom of Great Britain and Northern Ireland", "UK")
hospital_beds["Country"] = hospital_beds["Country"].replace("Republic of Korea", "South Korea")
hospital_beds["Country"] = hospital_beds["Country"].replace("Venezuela (Bolivarian Republic of)", "Venezuela")
hospital_beds["Country"] = hospital_beds["Country"].replace('Iran (Islamic Republic of)', 'Iran')
hospital_beds["Country"] = hospital_beds["Country"].replace('Russian Federation', 'Russia')
hospital_beds["Country"] = hospital_beds["Country"].replace('Brunei Darussalam', 'Brunei')
hospital_beds["Country"] = hospital_beds["Country"].replace('Republic of Moldova', 'Moldova')
hospital_beds["Country"] = hospital_beds["Country"].replace('United Republic of Tanzania', 'Tanzania')
hospital_beds["Country"] = hospital_beds["Country"].replace("Lao People's Democratic Republic", 'Laos')
hospital_beds["Country"] = hospital_beds["Country"].replace("Myanmar", 'Burma')
hospital_beds["Country"] = hospital_beds["Country"].replace("Congo", 'Congo')
hospital_beds["Country"] = hospital_beds["Country"].replace("Côte d'Ivoire", "Cote d'Ivoire")
hospital_beds["Country"] = hospital_beds["Country"].replace("State of Palestine", "West Bank/Gaza")
hospital_beds['Country'] = hospital_beds["Country"].replace("Bosnia and Herzegovina","Bosnia & H.")
hospital_beds['Country'] = hospital_beds["Country"].replace("North Macedonia","N. Macedonia")
hospital_beds['Country'] = hospital_beds["Country"].replace("Trinidad and Tobago","Trinidad & T.")
hospital_beds['Country'] = hospital_beds["Country"].replace("Antigua and Barbuda","Antigua & B.")
hospital_beds['Country'] = hospital_beds["Country"].replace("Saint Vincent and the Grenadines","St Vincent & G.")
hospital_beds['Country'] = hospital_beds["Country"].replace("Saint Kitts and Nevis",'St Kitts')
hospital_beds['Country'] = hospital_beds["Country"].replace("Dominican Republic",'Dominican Rep')
hospital_beds['Country'] = hospital_beds["Country"].replace("Sao Tome and Principe",'Sao Tome & P.')
hospital_beds['Country'] = hospital_beds["Country"].replace('United Arab Emirates','UAE')
hospital_beds['Country'] = hospital_beds["Country"].replace('Papua New Guinea','Papua N.G.')

## Some countries are not in the demographics datset because they have not performed a census within the specified period (at least 2007), so we will remove countries from the covid dataset which are not in the demographics dataset

In [20]:
# geting unique entries for country names in each dataframe
unique_covid_countries = daily['Country_Region'].unique()
unique_demographic_countries = demographics['Country or Area'].unique()
unique_demographic_countries.sort()

# we want to identify all of the copuntries with covid reporting that are not in (or named the same) in the demographics dataset

correct_name = list(set(unique_covid_countries) - set(unique_demographic_countries))
correct_name.sort()

## The remaining countries do not have data since 2000 and will be dropped from the dataframe.

In [21]:
# drop all countries from daily dataframe which do not have demographic records since 2007
daily_updated = daily[~daily['Country_Region'].isin(correct_name)]

In [22]:
# dropping all countries from the demographics dataframe which do not appear in the daily dataframe
demo_drop = list(set(unique_demographic_countries) - set(unique_covid_countries))
demo_drop.sort()

## Now we can confirm that the dataframe countries match by checking the unique values against each other.

In [23]:
demographics_updated = demographics[~demographics['Country or Area'].isin(demo_drop)]
sorted_daily = daily_updated['Country_Region'].unique()
sorted_daily.sort()
sorted_demo = demographics_updated['Country or Area'].unique()
sorted_demo.sort()


# checking output to see if names match
sorted_daily == sorted_demo

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

## We can make a universal country list as they are all the same now

In [24]:
country_list = sorted_daily

## Getting the most recent year for each country's demographics data

In [25]:
demographics_max_years = demographics_updated.groupby(['Country or Area'])['Year'].max()

## We want to make a data frame that only has data with the most recent years for each country

In [26]:
# creating a list of tuples containing the country and corresponding year of most recent data
zippedlist = list(zip(country_list, demographics_max_years[0:]))

# initializing a dataframe with the first values from the above list
demographics_consolidated = demographics_updated.loc[(demographics_updated['Country or Area'] == zippedlist[0][0]) & (demographics_updated['Year'] == zippedlist[0][1]), :]

# looping through the other countries and years and appending to the above dataframe
for country,year in zippedlist[1:]:
    df_temp = demographics_updated.loc[(demographics_updated['Country or Area'] == country) & (demographics_updated['Year'] == year), :]
    demographics_consolidated = demographics_consolidated.append(df_temp)

## We now need to consolidate age values into appropriate brackets for coivd-19 mortality analysis. 

## Beginning by checking the unique values present in 'Age"

In [27]:
demographics_consolidated.Age.unique()

array(['Total', '0', '0 - 4', '1', '1-4', '2', '3', '4', '5', '5-9', '6',
       '7', '8', '9', '10', '10-14', '11', '12', '13', '14', '15',
       '15 - 19', '16', '17', '18', '19', '20', '20 - 24', '21', '22',
       '23', '24', '25', '25 - 29', '26', '27', '28', '29', '30',
       '30 - 34', '31', '32', '33', '34', '35', '35 - 39', '36', '37',
       '38', '39', '40', '40 - 44', '41', '42', '43', '44', '45',
       '45 - 49', '46', '47', '48', '49', '50', '50 - 54', '51', '52',
       '53', '54', '55', '55 - 59', '56', '57', '58', '59', '60',
       '60 - 64', '61', '62', '63', '64', '65', '65 - 69', '66', '67',
       '68', '69', '70', '70 - 74', '71', '72', '73', '74', '75',
       '75 - 79', '76', '77', '78', '79', '80', '80 - 84', '81', '82',
       '83', '84', '85 +', '85', '85 - 89', '86', '87', '88', '89', '90',
       '90 - 94', '91', '92', '93', '94', '95 +', '95', '95 - 99', '96',
       '97', '98', '99', '100 +', '100', '80 +', '90 +', '100 - 104',
       '101', '102', '1

## As we can see there are ages range brackets that need to be converted to a single number (mean will do). Many of these age range brackets result in double counting (e.g., 0,1,2,3,4 are counted and then 0-4 is also counted for some countries)

## We need to replace the age ranges, then drop all duplicate ages from each country to ensure no double counting.

## We can't simply delete all entries containing a '-' because some countries only have age range data.

In [28]:
# function to convert a str age range into a str average of the age range
def change_vals(age):
    # keep all numbers that aren't a range
    if '-' not in age:
        return age
    #convert all ranges
    else:
        # split ranges into a list of characters
        age_fix = list(age.split('-'))
        # convert each char to int
        age_fix = [int(i) for i in age_fix]
        # get the mean of the list of ints
        age_fix = int(np.mean(age_fix))
        # convert mean back into str
        age_fix = str(age_fix)
        return age_fix

# Apply the change_vals function to the 'Age' column
demographics_consolidated.Age = demographics_consolidated.Age.apply(lambda x: change_vals(x))

# drop duplicate rows based on same country and age value, keeping the last value
demographics_consolidated = demographics_consolidated.drop_duplicates(subset=['Country or Area', 'Age'], keep='last')

## Rechecking unique age values, there should be no ranges.

In [29]:
demographics_consolidated.Age.unique()

array(['Total', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85 +', '85', '86',
       '87', '88', '89', '90', '91', '92', '93', '94', '95 +', '95', '96',
       '97', '98', '99', '100 +', '100', '80 +', '90 +', '101', '102',
       '103', '104', '105', '106', '107', '108', '109', '110 +', '75 +',
       '98 +'], dtype=object)

## Manually assigning brackets

In [30]:
under_30 = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29']

thirties = ['30', '31', '32', '33', '34', '35', '36', '37', '38', '39']

forties = ['40', '41', '42', '43', '44', '45',  '46', '47', '48', '49']

fifties = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59']

sixties = ['60', '61', '62', '63', '64', '65', '66', '67', '68', '69','65 +']

seventies = ['70', '71', '72', '73', '74', '75', '76', '77', '78', '79','75 +','70 +']

eighties_plus = ['80', '81', '82', '83', '84', '85 +', '85', '86', '87', '88', '89', 
                 '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100 +', 
                 '95 +', '100', '101', '102', '103', '104', '105', '106', '107', 
                 '108', '109', '110 +', '80 +', '98 +', '99 +','90 +']

## We need to apply the new age brackets to each country and consolidate and sum all the values for each age range

In [31]:
# Applying age brackets
demographics_consolidated.Age[demographics_consolidated['Age'].isin(under_30)] = '<30'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(thirties)] = '30s'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(forties)] = '40s'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(fifties)] = '50s'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(sixties)] = '60s'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(seventies)] = '70s'
demographics_consolidated.Age[demographics_consolidated['Age'].isin(eighties_plus)] = '80+'

#reseting the index
demographics_consolidated = demographics_consolidated.reset_index(drop=True)

# creating a column to sums the totals of all values which belong to the same age bracket (e.g., sum all values for <30 so that there is only one value for <30)
demographics_consolidated['New_values'] = demographics_consolidated.groupby(['Country or Area','Age'])['Value'].transform('sum')

# drop all duplicate rows for a combination of country and age (this is because there may be some double counting)
demographics_consolidated = demographics_consolidated.drop_duplicates(subset=['Country or Area', 'Age', 'Year', 'New_values'], keep='last')

# drop un-needing columns
demographics_final = demographics_consolidated.drop(columns = {'Value', 'Year'})

# rename columns for consistency with daily reports dataframe
demographics_final = demographics_final.rename(columns={'New_values': 'Values','Country or Area':'Country_Region'})

# reseting the index
demographics_final = demographics_final.reset_index(drop = True)

C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

## We want to consolidate the states/provinces in the daily report dataframe into countries and clean it up.

In [32]:
# create new rows containing the sums of all rows per country
daily_updated['Total_confirmed'] = daily_updated.groupby(['Country_Region'])['Confirmed'].transform('sum')
daily_updated['Total_deaths'] = daily_updated.groupby(['Country_Region'])['Deaths'].transform('sum')
daily_updated['Total_recovered'] = daily_updated.groupby(['Country_Region'])['Recovered'].transform('sum')

# drop rows with duplicate countries
daily_consolidated = daily_updated.drop_duplicates(subset=['Country_Region'])

# drop columns that are not required
daily_consolidated = daily_consolidated.drop(columns=['FIPS','Admin2','Province_State','Confirmed', 'Deaths', 'Recovered', 'Active', 'Combined_Key'])

# Renaming the new columns to old format
daily_final = daily_consolidated.rename(columns={'Total_confirmed': 'Confirmed', 'Total_deaths': 'Deaths' , 'Total_recovered':'Recovered'})

# Reseting the dataframe index
daily_final.reset_index(drop = True, inplace = True)

daily_final = daily_final.sort_values(["Country_Region"], ascending = (True))

C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# Checking our new dataframes:

In [33]:
demographics_final = demographics_final[~demographics_final['Age'].str.contains('Total')]
demographics_final.head()

,Country_Region,Age,Values
1,Albania,<30,1202786.00
2,Albania,30s,359276.00
3,Albania,40s,346046.00
4,Albania,50s,399874.00
5,Albania,60s,304201.00


In [34]:
daily_final.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered
8,Albania,2020-04-15 22:56:32,41.15,20.17,494,25,251
9,Algeria,2020-04-15 22:56:32,28.03,1.66,2160,336,708
10,Andorra,2020-04-15 22:56:32,42.51,1.52,673,33,169
11,Angola,2020-04-15 22:56:32,-11.20,17.87,19,2,5
12,Antigua & B.,2020-04-15 22:56:32,17.06,-61.80,23,2,3


## We are going to want to combine the demographics data into the daily dataframe so that we can have one dataframe with all of the information, we will call this datframe 'final'

## As the daily_final and demographics_final dataframes have the same number of countries and the same ordering of countries, we can simply sort the age brackets and use each bracket as a list which we can add to our final dataframe as a column

In [35]:
# using daily_final as a base
final = daily_final

# create columns for each age bracket from grouped age brackets in the demographics dataframe. 
final['<30'] = list(demographics_final.groupby("Age").get_group('<30').Values)
final['30s'] = list(demographics_final.groupby("Age").get_group('30s').Values)
final['40s'] = list(demographics_final.groupby("Age").get_group('40s').Values)
final['50s'] = list(demographics_final.groupby("Age").get_group('50s').Values)
final['60s'] = list(demographics_final.groupby("Age").get_group('60s').Values)
final['70s'] = list(demographics_final.groupby("Age").get_group('70s').Values)
final['80+'] = list(demographics_final.groupby("Age").get_group('80+').Values)

# Create a total population from the columns instead of from the data to ensure they add to 100%
final['Total_pop'] = daily_final['<30']+daily_final['30s']+daily_final['40s']+daily_final['50s']+daily_final['60s']+daily_final['70s']+daily_final['80+']

# creating % columns for each age bracket
final['<30_%'] = daily_final['<30'] / daily_final['Total_pop']*100
final['30s_%'] = daily_final['30s'] / daily_final['Total_pop']*100
final['40s_%'] = daily_final['40s'] / daily_final['Total_pop']*100
final['50s_%'] = daily_final['50s'] / daily_final['Total_pop']*100
final['60s_%'] = daily_final['60s'] / daily_final['Total_pop']*100
final['70s_%'] = daily_final['70s'] / daily_final['Total_pop']*100
final['80+_%'] = daily_final['80+'] / daily_final['Total_pop']*100

# getting weighted average of mortality based on the population brackets of each country
final['est_death_rate_%'] = ( 
                                     (daily_final['<30_%'] * mortality_by_age[0] )
                                    +(daily_final['30s_%'] * mortality_by_age[1] )
                                    +(daily_final['40s_%'] * mortality_by_age[2] )
                                    +(daily_final['50s_%'] * mortality_by_age[3] )
                                    +(daily_final['60s_%'] * mortality_by_age[4] )
                                    +(daily_final['70s_%'] * mortality_by_age[5] )
                                    +(daily_final['80+_%'] * mortality_by_age[6] ) 
                                    ) /100

# getting actual deaths per confirmed cases as recorded by each country
final['deaths/confirmed_%'] = final['Deaths']/final['Confirmed']*100

# getting deaths/million as a control metric
final['deaths/10k'] = (final['Deaths']/(final['Total_pop']/10000))

# Adding a column for 60+ as this is the at risk group
final['60+'] = final['60s']+final['70s']+final['80+']

# adding a 60+% column
final['60+%'] = final['60+']/final['Total_pop']*100

# adding population as a percentage of china's ie: china = 1, usa = .3
final['relative_pop'] = final['Total_pop']/float(final[final['Country_Region']=='China']['Total_pop'])

final = daily_final.reset_index(drop=True)

In [36]:
# adding population as a percentage of china's ie: china = 1, usa = .3
demographics_final['relative_pop'] = demographics_final.groupby(['Country_Region'])['Values'].transform('sum')/float(final[final['Country_Region']=='China']['Total_pop'])

## Checking final output

## Writing to filesa

In [37]:
demographics_final.to_csv('demographics_final.csv',index=False)
demographics_final.to_csv('covidapp/demographics_final.csv',index=False)
daily_final.to_csv('daily_final.csv',index=False)
final.to_csv('final.csv',index=False)

## I additionally want to make an R-friendly version of the final dataframe, so I will have to change some column names to match R syntax.

In [38]:
pylist = list(final.columns)
rlist = ["country", "update", "lat", "long", "confirmed", "deaths", "recovered", 
         "under30", "thirties", "fourties", "fifties", "sixties", "seventies", 
         "eighties", "total_pop", "under30_frac", "thirties_frac", "fourties_frac", 
         "fifties_frac", "sixties_frac", "seventies_frac", "eighties_frac", 
         "est_death_percent", "deaths_per_conf_perc", "deaths_per_10k", "over_sixty", "over_sixty_perc", 'relative_pop']

colnames = dict(zip(pylist,rlist))

finalr = final.rename(columns=colnames)

finalr.to_csv('covidapp/finalr.csv',index=False)

## Just a quick extra file which consolidates the 60 70 and 80 agegroups into a single over60 age group.

In [39]:
demographics_over60 = demographics_final

# Applying age brackets
demographics_over60.Age[demographics_over60['Age'].isin(['60s'])] = '60+'
demographics_over60.Age[demographics_over60['Age'].isin(['70s'])] = '60+'
demographics_over60.Age[demographics_over60['Age'].isin(['80+'])] = '60+'

#reseting the index
demographics_over60 = demographics_over60.reset_index(drop=True)

# creating a column to sums the totals of all values which belong to the same age bracket (e.g., sum all values for <30 so that there is only one value for <30)
demographics_over60['New_values'] = demographics_over60.groupby(['Country_Region','Age'])['Values'].transform('sum')

# drop all duplicate rows for a combination of country and age (this is because there may be some double counting)
demographics_over60 = demographics_over60.drop_duplicates(subset=['Country_Region', 'Age', 'New_values'], keep='last')

# drop un-needing columns
demographics_over60 = demographics_over60.drop(columns = {'Values'})

# rename columns for consistency with daily reports dataframe
demographics_over60 = demographics_over60.rename(columns={'New_values': 'Values'})

# reseting the index
demographics_over60 = demographics_over60.reset_index(drop = True)

demographics_over60.to_csv('covidapp/demographics_over60.csv',index=False)

C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\darra\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
finalr_choropleth_match = finalr

shapefile_countries = ["Aruba"         ,                           "Afghanistan"               ,               "Angola"                 ,                 
"Anguilla"                ,                 "Albania"                     ,             "Aland"                   ,                
"Andorra"               ,                  "United Arab Emirates"              ,       "Argentina"                 ,              
"Armenia"                       ,           "American Samoa"                ,           "Antarctica"                ,              
"Ashmore and Cartier Islands"      ,        "French Southern and Antarctic Lands"   ,   "Antigua and Barbuda"        ,             
"Australia"         ,                       "Austria"           ,                       "Azerbaijan"                    ,          
"Burundi"            ,                      "Belgium"            ,                      "Benin"                          ,         
"Burkina Faso"        ,                     "Bangladesh"          ,                     "Bulgaria"                        ,        
"Bahrain"              ,                    "The Bahamas"           ,                   "Bosnia and Herzegovina"           ,       
"Saint Barthelemy"      ,                   "Belarus"                ,                  "Belize"                            ,      
"Bermuda"                ,                  "Bolivia"                 ,                 "Brazil"                             ,     
"Barbados"                ,                 "Brunei"                   ,                "Bhutan"                              ,    
"Botswana"                 ,                "Central African Republic"  ,               "Canada"                               ,   
"Switzerland"               ,               "Chile"                      ,              "China"                                 ,  
"Ivory Coast"                ,              "Cameroon"                    ,             "Democratic Republic of the Congo"       , 
"Republic of Congo"           ,             "Cook Islands"                 ,            "Colombia"                                ,
"Comoros"                      ,            "Cape Verde"                    ,           "Costa Rica"                              ,
"Cuba"                          ,           "CuraÃ§ao"                       ,          "Cayman Islands"                          ,
"Northern Cyprus"                ,          "Cyprus"                          ,         "Czech Republic"                          ,
"Germany"                         ,         "Djibouti"                         ,        "Dominica"                                ,
"Denmark"                          ,        "Dominican Republic"                ,       "Algeria"                                 ,
"Ecuador"                           ,       "Egypt"                              ,      "Eritrea"                                 ,
"Spain"         ,                           "Estonia"                             ,     "Ethiopia"                                ,
"Finland"        ,                          "Fiji"                                 ,    "Falkland Islands"                        ,
"France"          ,                         "Faroe Islands"                         ,   "Federated States of Micronesia"          ,
"Gabon"            ,                        "United Kingdom"                         ,  "Georgia"                                 ,
"Guernsey"          ,                       "Ghana"                        ,            "Guinea"                                  ,
"Gambia"             ,                      "Guinea Bissau"              ,              "Equatorial Guinea"                       ,
"Greece"              ,                     "Grenada"                     ,             "Greenland"                               ,
"Guatemala"            ,                    "Guam"                          ,           "Guyana"                                  ,
"Hong Kong S.A.R."      ,                   "Heard Island and McDonald Islands",        "Honduras"                                ,
"Croatia"                ,                  "Haiti"                             ,       "Hungary"                                 ,
"Indonesia"               ,                 "Isle of Man"                        ,      "India"                                   ,
"Indian Ocean Territories" ,                "British Indian Ocean Territory"      ,     "Ireland"                                 ,
"Iran"                      ,               "Iraq"    ,                                 "Iceland"                                 ,
"Israel"                     ,              "Italy"    ,                                "Jamaica"                                 ,
"Jersey"                      ,             "Jordan"    ,                               "Japan"                                   ,
"Siachen Glacier"              ,            "Kazakhstan" ,                              "Kenya"                                   ,
"Kyrgyzstan"                    ,           "Cambodia"    ,                             "Kiribati"                                ,
"Saint Kitts and Nevis"          ,          "South Korea"  ,                            "Kosovo"                                  ,
"Kuwait"                          ,         "Laos"          ,                           "Lebanon"                                 ,
"Liberia"                          ,        "Libya"          ,                          "Saint Lucia"                             ,
"Liechtenstein"                     ,       "Sri Lanka"       ,                         "Lesotho"                                 ,
"Lithuania"                          ,      "Luxembourg"       ,                        "Latvia"                                  ,
"Macao S.A.R"                         ,     "Saint Martin"      ,                       "Morocco"                                 ,
"Monaco"       ,                            "Moldova"            ,                      "Madagascar"                              ,
"Maldives"      ,                           "Mexico"              ,                     "Marshall Islands"                        ,
"Macedonia"      ,                          "Mali"                 ,                    "Malta"                                   ,
"Myanmar"         ,                         "Montenegro"            ,                   "Mongolia"                                ,
"Northern Mariana Islands"   ,              "Mozambique"             ,                  "Mauritania"                              ,
"Montserrat"        ,                       "Mauritius"               ,                 "Malawi"                                  ,
"Malaysia"           ,                      "Namibia"                  ,                "New Caledonia"                           ,
"Niger"               ,                     "Norfolk Island"            ,               "Nigeria"                                 ,
"Nicaragua"            ,                    "Niue"                       ,              "Netherlands"                             ,
"Norway"                ,                   "Nepal"                       ,             "Nauru"                                   ,
"New Zealand"            ,                  "Oman"                         ,            "Pakistan"                                ,
"Panama"                  ,                 "Pitcairn Islands"              ,           "Peru"                                    ,
"Philippines"              ,                "Palau"                          ,          "Papua New Guinea"                        ,
"Poland"                    ,               "Puerto Rico"                     ,         "North Korea"                             ,
"Portugal"                   ,              "Paraguay"                         ,        "Palestine"                               ,
"French Polynesia"            ,             "Qatar"                             ,       "Romania"                                 ,
"Russia"                       ,            "Rwanda"                             ,      "Western Sahara"                          ,
"Saudi Arabia"                  ,           "Sudan"                               ,     "South Sudan"                             ,
"Senegal"                        ,          "Singapore"                            ,    "South Georgia and South Sandwich Islands",
"Saint Helena"                    ,         "Solomon Islands"                       ,   "Sierra Leone"                            ,
"El Salvador"                      ,        "San Marino"                             ,  "Somaliland"                              ,
"Somalia"                           ,       "Saint Pierre and Miquelon" ,               "Republic of Serbia"                      ,
"Sao Tome and Principe"              ,      "Suriname"                   ,              "Slovakia"                                ,
"Slovenia"                            ,     "Sweden"                      ,             "Swaziland"                               ,
"Sint Maarten"                         ,    "Seychelles"                   ,            "Syria"                                   ,
"Turks and Caicos Islands"              ,   "Chad"                          ,           "Togo"                                    ,
"Thailand"     ,                            "Tajikistan"                     ,          "Turkmenistan"                            ,
"East Timor"    ,                           "Tonga"                           ,         "Trinidad and Tobago"                     ,
"Tunisia"        ,                          "Turkey"                           ,        "Taiwan"                                  ,
"United Republic of Tanzania"  ,            "Uganda"                            ,       "Ukraine"                                 ,
"Uruguay"                       ,           "United States of America"           ,      "Uzbekistan"                              ,
"Vatican"                        ,          "Saint Vincent and the Grenadines"    ,     "Venezuela"                               ,
"British Virgin Islands"          ,         "United States Virgin Islands"         ,    "Vietnam"                                 ,
"Vanuatu"                          ,        "Wallis and Futuna"                     ,   "Samoa"                                   ,
"Yemen"                             ,       "South Africa"                           ,  "Zambia"                                  ,
"Zimbabwe" ]

In [41]:
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("UK","United Kingdom")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("Bosnia & H.","Bosnia and Herzegovina")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("West Bank/Gaza","Palestine")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("N. Macedonia","Macedonia")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("Trinidad & T.","Trinidad and Tobago")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("Antigua & B.","Antigua and Barbuda")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("St Vincent & G.","Saint Vincent and the Grenadines")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('St Kitts',"Saint Kitts and Nevis")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Dominican Rep',"Dominican Republic")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Sao Tome & P.',"Sao Tome and Principe")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('UAE','United Arab Emirates')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Papua N.G.','Papua New Guinea')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('US','United States of America')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Bahamas','The Bahamas')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("Cote d'Ivoire",'Ivory Coast')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Timor-Leste','East Timor')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Cabo Verde','Cape Verde')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Czechia','Czech Republic')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Guinea-Bissau','Guinea Bissau')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Tanzania',"United Republic of Tanzania")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace("Korea, South","South Korea")
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Congo','Republic of Congo')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Burma','Myanmar')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Eswatini','Swaziland')
finalr_choropleth_match['country'] = finalr_choropleth_match["country"].replace('Serbia','Republic of Serbia')
list(set(list(finalr_choropleth_match.country)) - set(shapefile_countries))

[]

In [42]:
list(set(shapefile_countries)
     -set(list(finalr_choropleth_match.country)))

['Marshall Islands',
 'Jersey',
 'Kiribati',
 'Djibouti',
 'French Southern and Antarctic Lands',
 'Aland',
 'Antarctica',
 'Greenland',
 'Turks and Caicos Islands',
 'Anguilla',
 'French Polynesia',
 'Puerto Rico',
 'Solomon Islands',
 'South Sudan',
 'Northern Cyprus',
 'Bermuda',
 'Afghanistan',
 'Saint Barthelemy',
 'Northern Mariana Islands',
 'Falkland Islands',
 'Guam',
 'Vanuatu',
 'Eritrea',
 'Heard Island and McDonald Islands',
 'Sint Maarten',
 'Pitcairn Islands',
 'Wallis and Futuna',
 'Saint Martin',
 'CuraÃ§ao',
 'Jamaica',
 'North Korea',
 'Aruba',
 'Fiji',
 'New Caledonia',
 'Syria',
 'Madagascar',
 'Taiwan',
 'Siachen Glacier',
 'Guatemala',
 'Haiti',
 'Samoa',
 'Central African Republic',
 'Isle of Man',
 'Guernsey',
 'Western Sahara',
 'Vatican',
 'Cook Islands',
 'Norfolk Island',
 'Comoros',
 'Montserrat',
 'Palau',
 'British Virgin Islands',
 'Hong Kong S.A.R.',
 'Yemen',
 'Saint Pierre and Miquelon',
 'Tonga',
 'Tajikistan',
 'Federated States of Micronesia',
 'T

In [43]:
finalr_choropleth_match = finalr_choropleth_match.drop(columns={
       'under30', 'thirties', 'fourties', 'fifties', 'sixties', 'seventies',
       'eighties', 'total_pop', 'under30_frac', 'thirties_frac',
       'fourties_frac', 'fifties_frac', 'sixties_frac', 'seventies_frac',
       'eighties_frac', 
       'deaths_per_10k', 'over_sixty', 'relative_pop'})
finalr_choropleth_match.to_csv('covidapp/finalr_choropleth_match.csv',index=False)

In [44]:
finalr_choropleth_match

,country,update,lat,long,confirmed,deaths,recovered,est_death_percent,deaths_per_conf_perc,over_sixty_perc
0,Albania,2020-04-15 22:56:32,41.15,20.17,494,25,251,0.70,5.06,19.59
1,Algeria,2020-04-15 22:56:32,28.03,1.66,2160,336,708,0.30,15.56,7.50
2,Andorra,2020-04-15 22:56:32,42.51,1.52,673,33,169,0.75,4.90,19.41
3,Angola,2020-04-15 22:56:32,-11.20,17.87,19,2,5,0.15,10.53,3.76
4,Antigua and Barbuda,2020-04-15 22:56:32,17.06,-61.80,23,2,3,0.49,8.70,13.34
...,...,...,...,...,...,...,...,...,...,...
157,Venezuela,2020-04-15 22:56:32,6.42,-66.59,197,9,111,0.40,4.57,10.98
158,Vietnam,2020-04-15 22:56:32,14.06,108.28,267,0,171,0.45,0.00,11.86
159,Palestine,2020-04-15 22:56:32,31.95,35.23,374,2,63,0.18,0.53,4.52
160,Zambia,2020-04-15 22:56:32,-13.13,27.85,48,2,30,0.15,4.17,3.93
